# Accessing Data

In [ ]:
# import numpy and pandas
import numpy as np
import pandas as pd

## Reading a CSV into a DataFrame

In [ ]:
# read in msft.csv into a DataFrame
msft = pd.read_csv("data/msft.csv")
msft[:5]

## Specifying the index column when reading a CSV file

In [ ]:
# use column 0 as the index
msft = pd.read_csv("data/msft.csv", index_col=0)
msft[:5]

## Data type inference and specification

In [ ]:
# examine the types of the columns in this DataFrame
msft.dtypes

In [ ]:
# specify that the Volume column should be a float64
msft = pd.read_csv("data/msft.csv", 
                   dtype = { 'Volume' : np.float64})
msft.dtypes

## Specifying column names

In [ ]:
# specify a new set of names for the columns
# all lower case, remove space in Adj Close
# also, header=0 skips the header row
df = pd.read_csv("data/msft.csv", 
                 header=0,
                 names=['date', 'open', 'high', 'low', 'close', 'volume'])
df[:5]

## Specifying specific columns to load

In [ ]:
# read in data only in the Date and Close columns
# and index by the Date column
df2 = pd.read_csv("data/msft.csv", 
                  usecols=['Date', 'Close'], 
                  index_col=['Date'])
df2[:5]

## Saving a DataFrame to a CSV

In [ ]:
# save df2 to a new csv file
# also specify naming the index as date
df2.to_csv("data/msft_modified.csv", index_label='date')

## General field-delimited data

In [ ]:
# use read_table with sep=',' to read a CSV
df = pd.read_table("data/msft.csv", sep=',')
df[:5]

In [ ]:
# save as pipe delimited
df.to_csv("data/msft_piped.txt", sep='|')

## Handling messy data file

In [ ]:
# messy file
!head -n 6 data/msft2.csv # osx or Linux
# type data/msft2.csv # windows

In [ ]:
# read, but skip rows 0, 2 and 3
df = pd.read_csv("data/msft2.csv", skiprows=[0, 2, 3])
df[:5]

In [ ]:
# another messy file, with the mess at the end
!cat data/msft_with_footer.csv # osx or Linux
# type data/msft_with_footer.csv # windows

In [ ]:
# skip only two lines at the end
df = pd.read_csv("data/msft_with_footer.csv", 
                 skipfooter=2,
                 engine = 'python')
df

In [ ]:
# only process the first three rows
pd.read_csv("data/msft.csv", nrows=3)

In [ ]:
# skip 100 lines, then only process the next five
pd.read_csv("data/msft.csv", skiprows=100, nrows=5, 
            header=0,
            names=['date', 'open', 'high', 'low', 'close', 'vol']) 

## Reading and writing data in Excel format

In [ ]:
# read excel file, it require `openpyxl` package
# only reads first sheet (msft in this case)
df = pd.read_excel("data/stocks.xlsx")
df[:5]

In [ ]:
# read from the aapl worksheet
aapl = pd.read_excel("data/stocks.xlsx", sheet_name='aapl')
aapl[:5]

In [ ]:
# save to an .XLS file, in worksheet 'Sheet1'
df.to_excel("data/stocks2.xls")

In [ ]:
# write making the worksheet name MSFT
df.to_excel("data/stocks_msft.xls", sheet_name='MSFT')

In [ ]:
# write multiple sheets
# requires use of the ExcelWriter class
from pandas import ExcelWriter
with ExcelWriter("data/all_stocks.xls") as writer:
    aapl.to_excel(writer, sheet_name='AAPL')
    df.to_excel(writer, sheet_name='MSFT')

In [ ]:
# write to xlsx
df.to_excel("data/msft2.xlsx")

## Reading and writing JSON files

In [ ]:
# wirite the excel data to a JSON file
df[:5].to_json("data/stocks.json")
!cat data/stocks.json # osx or Linux
#type data/stocks.json # windows

In [ ]:
# read data in from JSON
df_from_json = pd.read_json("https://raw.githubusercontent.com/PacktPublishing/Learning-Pandas-Second-Edition/master/data/stocks.json")
df_from_json[:5]

## Reading and writing HTML files

In [ ]:
# the URL to read, require `lxml` package
url = "http://www.fdic.gov/bank/individual/failed/banklist.html"
# read it
banks = pd.read_html(url)

In [ ]:
# examine a subset of the first table read
banks[0][0:5].iloc[:,0:2]

In [ ]:
# read the stock data
df = pd.read_excel("data/stocks.xlsx")
# write the first two rows to HTML
df.head(2).to_html("data/stocks.html")
# check the first 28 lines of the output
!head -n 10 data/stocks.html # max or Linux
# type data/stocks.html # window, but prints the entire file

## Accessing CSV data on the web

In [ ]:
# read csv directly from Yahoo! Finance from a URL
msft_hist = pd.read_csv(
    "https://raw.githubusercontent.com/PacktPublishing/Learning-Pandas-Second-Edition/master/data/msft.csv")
msft_hist[:5]

## Writing to SQL databases

In [ ]:
# reference SQLite
import sqlite3

# read in the stock data from CSV
msft = pd.read_csv("data/msft.csv")
msft["Symbol"]="MSFT"
aapl = pd.read_csv("data/aapl.csv")
aapl["Symbol"]="AAPL"

# create connection
connection = sqlite3.connect("data/stocks.sqlite")
# .to_sql() will create SQL to store the DataFrame
# in the specified table.  if_exists specifies
# what to do if the table already exists
msft.to_sql("STOCK_DATA", connection, if_exists="replace")
aapl.to_sql("STOCK_DATA", connection, if_exists="append")

# commit the SQL and close the connection
connection.commit()
connection.close()

## Reading from SQL databases

In [ ]:
# connect to the database file
connection = sqlite3.connect("data/stocks.sqlite")

# query all records in STOCK_DATA
# returns a DataFrame
# inde_col specifies which column to make the DataFrame index
stocks = pd.io.sql.read_sql("SELECT * FROM STOCK_DATA;", 
                             connection, index_col='index')

# close the connection
connection.close()

# report the head of the data retrieved
stocks[:5]

In [ ]:
# open the connection
connection = sqlite3.connect("data/stocks.sqlite")
# construct the query string
query = "SELECT * FROM STOCK_DATA WHERE " + \
        "Volume>29200100 AND Symbol='MSFT';"
# execute and close connection
items = pd.io.sql.read_sql(query, connection, index_col='index')
connection.close()
# report the query result
items